## Import required modules

In [1]:
import numpy as np
import pandas as pd 
from numba import jit, njit
import os
import sys
import zarr
import pickle
import time
import tables
os.chdir('../../')
sys.path.append('tools/')
from SliceData import SliceData
from SliceStore import SliceStore

from lookup_functions import *

1  was not stored in memory, it is going to be loaded from disk
allen mouse atlas (res. 25um)
From: http://www.brain-map.org (Wang et al 2020, https://doi.org/10.1016/j.cell.2020.04.007 )


## Load data

In [2]:
#Load slice 1 
slice = SliceData(1)

#print size used by each array in mb
print(round(slice.array_pixel_indexes_high_res.nbytes/ 1024 / 1024,2))
print(round(slice.array_spectra_high_res.nbytes/ 1024 / 1024,2))
print(round(slice.array_averaged_mz_intensity_low_res.nbytes/ 1024 / 1024,2))
print(round(slice.array_averaged_mz_intensity_high_res.nbytes/ 1024 / 1024,2))
print(round(slice.lookup_table_spectra_high_res.nbytes/ 1024 / 1024,2))
print(round(slice.cumulated_image_lookup_table_high_res.nbytes/ 1024 / 1024,2))
print(round(slice.lookup_table_averaged_spectrum_high_res.nbytes/ 1024 / 1024,2))



0.57
735.72
0.37
6.87
56.75
56.75
0.01


### Play with Pickle

In [ ]:
#Pickle all slices for future comparison
pickle_all = True
if pickle_all:
    slice_store = SliceStore()
    SliceStore.pickleAllSlices()

In [4]:
path = "data/hdf5/slices2.hdf5"
a = time.time()
read_hdf5_file = tables.open_file(path, mode='r' )
hdf5_data = read_hdf5_file.root['slice_30'][:]
read_hdf5_file.close()
b = time.time()
print(b-a)


#compare reading times
path = "data/pickled_data/raw_data/"

a = time.time()
slice_index = 2
name_fig = "data_slice_" + str(slice_index) + ".pickle"
slice = pickle.load(open(path + name_fig, "rb"))
b = time.time()

print(b-a)

a = time.time()
slice_index = 30
name_fig = "data_slice_" + str(slice_index) + ".pickle"
slice = pickle.load(open(path + name_fig, "rb"))
b = time.time()

print(b-a)




1.820162057876587
0.2095470428466797
1.0304827690124512


### Play with pytables

In [2]:
path = "data/hdf5/slices.hdf5"
hdf5_file = tables.open_file(path, mode='w')
for index_slice in range(29,31):
    slice = SliceData(index_slice, load_from_pytables=False)
    hdf5_file.create_array(hdf5_file.root, 'slice_' + str(index_slice), slice.array_spectra_high_res)
hdf5_file.close()

/opt/miniconda3/lib/python3.9/site-packages/tables/leaf.py:402: PerformanceWarning:

The Leaf ``/slice_29`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.

/opt/miniconda3/lib/python3.9/site-packages/tables/leaf.py:402: PerformanceWarning:

The Leaf ``/slice_30`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may wan

In [10]:
hdf5_file = tables.open_file(path, mode='r')
print(hdf5_file.list_nodes
hdf5_file.close()

/slice_30 (Array(2, 307003267)) ''


In [21]:
hdf5_file.close()

In [40]:
path = "data/hdf5/slices.hdf5"

a = time.time()
for x in range(10):
    read_hdf5_file = tables.open_file(path, mode='r' )
    hdf5_data = read_hdf5_file.root['slice_30'][0,1000:2000]
    read_hdf5_file.close()
b = time.time()
print( (b-a)/10)


0.0016515254974365234


In [ ]:
#see if the whole file is loaded
np.sum(hdf5_data)

In [ ]:
#try compression
path = "data/hdf5/slice_compressed.hdf5"
hdf5_file = tables.open_file(path, mode='w')
filters = tables.Filters(complevel=5, complib='blosc:zstd')
data_storage = hdf5_file.create_carray(hdf5_file.root, 'data', tables.Atom.from_dtype(slice.array_spectra_high_res.dtype),  shape=slice.array_spectra_high_res.T.shape,filters=filters)
data_storage[:] = slice.array_spectra_high_res.T
hdf5_file.close()

In [3]:
#check if a basic plot is doable in acceptable time... First load lookup tables through slice object
currentSlice = SliceData(30, load_from_pytables=False)

In [ ]:
#how long does it take to take the transpose?
a = time.time()
aa = currentSlice.array_spectra_high_res.T
b = time.time()
print(b-a)

In [11]:
a = time.time()
path = "data/hdf5/slices.hdf5"
read_hdf5_file = tables.open_file(path, mode='r' )
hdf5_data = read_hdf5_file.root['slice_30'][:]
for idx_pix in range(currentSlice.array_pixel_indexes_high_res.shape[0]):
    #test = hdf5_data[currentSlice.array_pixel_indexes_high_res[idx_pix, 0]: currentSlice.array_pixel_indexes_high_res[idx_pix, 1]+1,:]
    test = hdf5_data[currentSlice.array_pixel_indexes_high_res[idx_pix, 0]: currentSlice.array_pixel_indexes_high_res[idx_pix, 0]+10,:]
read_hdf5_file.close()
b = time.time()
print(b-a)


1.6079881191253662


## Other tests (unfinished)

### Play with memmap

In [ ]:
fp = np.memmap('test.np', dtype='float32', mode='w+', shape=slice.array_spectra_high_res.shape) 
fp[:] = slice.array_spectra_high_res
print(slice.array_spectra_high_res.shape)

In [ ]:
fp = np.memmap('test.np', dtype='float32', mode='r', shape=(2, 96431659))
import plotly.graph_objects as go


### Play with zarr

In [ ]:
#register slices
for index_slice in range(30,31):
    path = "data/mmap/"
    array_spectra_high_res = SliceData(index_slice).array_spectra_high_res.T
    z1 = zarr.open(path + str(index_slice)+'.zarr', mode='w', shape=array_spectra_high_res.shape, chunks=(10000, -1), dtype='f4', compressor = None)
    z1[:] = array_spectra_high_res

In [ ]:
slice = SliceData(30, load_from_mmap=True)
slice.array_spectra_high_res.info

In [ ]:
a = time.time()
slice = SliceData(30, load_from_mmap=False)
b = time.time()
print(b-a)

In [ ]:
#load full slice
a = time.time()
np.sum(slice.array_spectra_high_res)
b = time.time()
print(b-a)

In [ ]:
slice_to_sum = slice.array_spectra_high_res[:]

In [ ]:
#load full slice
a = time.time()
np.sum(slice_to_sum)
b = time.time()
print(b-a)